In [60]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from tqdm import tqdm
from optuna import Trial
import optuna
from optuna.samplers import TPESampler

In [61]:
sample_df = pd.read_csv('train_data_final.csv', sep=',')

# 시간 순서대로 데이터 반영하기 위해서 거래년월일 정렬 필요
sample_df = sample_df.sort_values('계약날짜인코딩')
# sample_df = sample_df[sample_df.columns.difference(['동별 공원 갯수', '8학군', '동별지하철역수', '구별 교과학원 갯수', '계약날짜인코딩'])]
# sample_df = sample_df[sample_df.columns.difference(['전용면적','동별 공원 갯수', '동별지하철역수', '구별 교과학원 갯수', '계약날짜인코딩'])] #8학군
sample_df = sample_df.fillna(0)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237987 entries, 64104 to 208092
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   전용면적         237987 non-null  float64
 1   거래금액         237987 non-null  int64  
 2   층            237987 non-null  int64  
 3   건축년도         237987 non-null  int64  
 4   평당단가         237987 non-null  float64
 5   구            237987 non-null  object 
 6   동            237987 non-null  object 
 7   단지명브랜드       237987 non-null  object 
 8   계약날짜인코딩      237987 non-null  int64  
 9   구별 공원 갯수     237987 non-null  float64
 10  8학군          237987 non-null  int64  
 11  동별지하철역수      237987 non-null  int64  
 12  구별 교과학원 갯수   237987 non-null  int64  
 13  구별 대형마트 수    237987 non-null  float64
 14  구별 백화점 수     237987 non-null  float64
 15  구별 전체 마트 수   237987 non-null  float64
 16  구별 대학병원 병원수  237987 non-null  int64  
 17  구별 종합병원 병원수  237987 non-null  int64  
 18  단지명별 신축거래비율  237987 

In [62]:
# 동 레이블 인코딩
from tqdm import tqdm
dong_price = sample_df.groupby('동')['거래금액'].agg('mean').sort_values(ascending=False)
# 가격기준으로 동을 정렬한 리스트를 바탕으로 dong에 대해 라벨 인코딩 진행 - 477 it.
for i, d in tqdm(enumerate(list(dong_price.index)), total=len(dong_price)):
    sample_df.loc[sample_df['동'] == d, '동'] = i
    # test_dat.loc[test_df['동'] == d, '동'] = i
sample_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 330/330 [00:05<00:00, 56.02it/s]


,전용면적,거래금액,층,건축년도,평당단가,구,동,단지명브랜드,계약날짜인코딩,구별 공원 갯수,8학군,동별지하철역수,구별 교과학원 갯수,구별 대형마트 수,구별 백화점 수,구별 전체 마트 수,구별 대학병원 병원수,구별 종합병원 병원수,단지명별 신축거래비율
64104,57.8800,39500,5,2000,2252.073255,송파구,130,기타,0,144.0,0,0,655,4.0,2.0,6.0,2,16,0.110657
78845,84.9400,37700,3,1993,1464.680951,중랑구,265,두산|두산위브,0,11.0,0,3,174,6.0,0.0,6.0,3,12,0.127080
79128,84.7054,59500,11,2015,2318.034033,중랑구,265,하늘채,0,11.0,0,3,174,6.0,0.0,6.0,3,12,0.673469
15567,84.8400,52000,8,1996,2022.630835,관악구,202,삼성,0,21.0,0,3,253,1.0,1.0,2.0,2,8,0.021814
1983,84.2360,180000,19,2008,7051.616886,강남구,12,힐스테이트,0,132.0,0,3,1304,1.0,6.0,7.0,4,33,0.550668


In [63]:
sample_df = sample_df.astype({'동':'int'})
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237987 entries, 64104 to 208092
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   전용면적         237987 non-null  float64
 1   거래금액         237987 non-null  int64  
 2   층            237987 non-null  int64  
 3   건축년도         237987 non-null  int64  
 4   평당단가         237987 non-null  float64
 5   구            237987 non-null  object 
 6   동            237987 non-null  int32  
 7   단지명브랜드       237987 non-null  object 
 8   계약날짜인코딩      237987 non-null  int64  
 9   구별 공원 갯수     237987 non-null  float64
 10  8학군          237987 non-null  int64  
 11  동별지하철역수      237987 non-null  int64  
 12  구별 교과학원 갯수   237987 non-null  int64  
 13  구별 대형마트 수    237987 non-null  float64
 14  구별 백화점 수     237987 non-null  float64
 15  구별 전체 마트 수   237987 non-null  float64
 16  구별 대학병원 병원수  237987 non-null  int64  
 17  구별 종합병원 병원수  237987 non-null  int64  
 18  단지명별 신축거래비율  237987 

### 1. 무지성 인코딩

In [64]:

'''
    - 카테고리: 데이터 전처리
    - 개요: 무지성 인코딩 => 전체 컬럼 데이터 타입 확인 후 통째로 범주형 인코딩 처리
    - param: df
    - return: encoded columns dataframe
'''
def one_hot(df):
    # 컬럼의 이름 리스트로 뽑아오기
    columns_name_list = list(df.columns)

    # 컬럼마다 for문 반복
    for col_name in columns_name_list:
        # 만약 컬럼의 값 타입이 범주형이면
        if df[col_name].dtype == object or df[col_name].dtype == str:

            # # 컬럼의 유니크한 값을 리스트로 만들어둠
            col_items = df[col_name].unique().tolist()
            # print(col_items)

            onehot = OneHotEncoder(sparse=False)
            # print(col_name)
            onehot_encoded_arr = onehot.fit_transform(df[col_name].values.reshape(-1, 1))
            onehot_encoded_label = onehot.categories_[0]
            onehot_encoded_df = pd.DataFrame(onehot_encoded_arr, columns=onehot_encoded_label)
            # print(onehot_encoded_df)
            df.drop(col_name, axis=1, inplace=True)
            df = pd.concat([df, onehot_encoded_df], axis=1)
        else:
            # 종속(타겟) 컬럼만 인코딩 제외
            if col_name == '평당단가':
                continue
            scaled_label = [col_name]
            x = df[col_name].values.reshape(-1, 1) #returns a numpy array
            min_max_scaler = preprocessing.MinMaxScaler()
            x_scaled = min_max_scaler.fit_transform(x)
            scaled_df = pd.DataFrame(x_scaled, columns=scaled_label)

            df.drop(col_name, axis=1, inplace=True)
            df = pd.concat([df, scaled_df], axis=1)

    return df
'''
    - 카테고리: 데이터 전처리
    - 개요: 무지성 인코딩 => 새로운 데이터 컬럼 추가 시 별도 인코딩 결과만 출력할 때 사용 ㄱㄱ
    - param: df, col_name
    - return: encoded columns dataframe or series
'''
def encode_column(df, col_name):

    # column data type이 object 또는 str 즉 범주형일 경우
    # onehot 인코딩 수행된 데이터프레임 return
    if df[col_name].dtype == object or df[col_name].dtype == str:
        onehot = OneHotEncoder(sparse=False)

        onehot_encoded_arr = onehot.fit_transform(df[col_name].values.reshape(-1, 1))
        onehot_encoded_label = onehot.categories_[0]
        onehot_encoded_df = pd.DataFrame(onehot_encoded_arr, columns=onehot_encoded_label)

        return onehot_encoded_df

    # column data type이 나머지 타입일 경우
    # 해당 컬럼의 series return
    else:
        return df[col_name]

In [65]:
# 1-1 test one_hot function
preprocessed_train_df = one_hot(sample_df)
preprocessed_train_df

,평당단가,전용면적,거래금액,층,건축년도,강남구,강동구,강북구,강서구,관악구,...,구별 공원 갯수,8학군,동별지하철역수,구별 교과학원 갯수,구별 대형마트 수,구별 백화점 수,구별 전체 마트 수,구별 대학병원 병원수,구별 종합병원 병원수,단지명별 신축거래비율
0,5364.511692,0.173928,0.149293,0.044118,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.916667,1.0,0.142857,1.000000,0.166667,1.0,0.875,0.571429,1.000000,0.110657
1,4828.060523,0.272890,0.133717,0.014706,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.916667,1.0,0.142857,1.000000,0.166667,1.0,0.875,0.571429,1.000000,0.110657
2,5364.511692,0.272032,0.149293,0.000000,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.916667,1.0,0.142857,1.000000,0.166667,1.0,0.875,0.571429,1.000000,0.110657
3,5756.533700,0.272524,0.160676,0.014706,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.916667,1.0,0.142857,1.000000,0.166667,1.0,0.875,0.571429,1.000000,0.110657
4,6452.346308,0.270315,0.122334,0.058824,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.916667,1.0,0.142857,1.000000,0.166667,1.0,0.875,0.571429,1.000000,0.110657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237982,1410.601638,0.273036,0.047927,0.073529,0.700000,0.0,0.0,0.0,0.0,0.0,...,0.076389,0.0,0.142857,0.101749,1.000000,0.0,0.750,0.428571,0.322581,0.110657
237983,1269.794287,0.181498,0.024682,0.014706,0.700000,0.0,0.0,0.0,0.0,0.0,...,0.076389,0.0,0.142857,0.101749,1.000000,0.0,0.750,0.428571,0.322581,0.110657
237984,1513.445904,0.257164,0.046250,0.073529,0.766667,0.0,0.0,0.0,0.0,0.0,...,0.076389,0.0,0.142857,0.101749,1.000000,0.0,0.750,0.428571,0.322581,0.016495
237985,1805.330937,0.144890,0.059190,0.058824,0.766667,0.0,0.0,0.0,0.0,0.0,...,0.076389,0.0,0.142857,0.101749,1.000000,0.0,0.750,0.428571,0.322581,0.016495


In [66]:
# 인코딩된 전체 데이터 셋에서 train, test split하여 분할 시행
# fit 함수에서 사용할 예정 train(모델 훈련 및 도출), test(최종 검증 데이터)

# 인코딩된 데이터에서 data, target 분류
data = preprocessed_train_df[preprocessed_train_df.columns.difference(['평당단가', '거래금액'])]
target = preprocessed_train_df['평당단가'] 

train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.4, shuffle=False)

# train_x(data), train_y(target) => cross validation을 이용하여 교차 검증 및 점수 도출할 데이터
# test_x(data), test_y(target) => 향후 train data를 이용하여 생성된 최종 모형 평가하기 위한 테스트 데이터
# 무작위가 아닌 시간순 정렬 반영 위해 shuffle=False

len(train_x), len(train_y)

(142792, 142792)

### 2. 모델링

In [67]:
'''
    - 카테고리: 모델링
    - 개요: 머신러닝 모델링 수행 및 점수 도출
        - 교차 검증 방법으로 TimeSeriesSplit 수행
    - param: 

        1. model_tuple => ex. ('LR', LinearRegression())
        2. X_train, y_train, X_test, y_test

    - return: rmse
'''
def execute_modeling(model_tuple, data, target):

    name = model_tuple[0]
    model = model_tuple[1]

    # TimeSeries Cross validation 
    tscv = TimeSeriesSplit(n_splits=7)

    # 각 모델에 대하여 교차 검증한 결과 점수 확인
    # r^2, rmse 같이 도출 ㄱㄱ
    # scoring parameter option 어캐 줘야 함?
    score_list = []
    score_type = ['r2', 'neg_mean_squared_error']

    for s_type in score_type:

        scores = cross_val_score(model, data, target, cv=tscv, scoring=s_type)

        if s_type == 'neg_mean_squared_error':
            scores = np.sqrt(-scores)

        print(f'{name} average {s_type}: {scores.mean()}, score_list {s_type}: {scores}')

        score_list.append(scores.mean())

    rmse_score = score_list[1]

    return rmse_score

In [68]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

# 2. test execute_modeling function
preprocessed_train_df = one_hot(sample_df)
preprocessed_train_df = preprocessed_train_df.drop_duplicates()

model_list = [
#                 ('LR', LinearRegression()), 
                ('RF', RandomForestRegressor(n_estimators=5)),
                ('model_xgb', xgb.XGBRegressor()),
                ('model_lgb', lgb.LGBMRegressor())
            ]
for model_tuple in tqdm(model_list):
    execute_modeling(model_tuple, train_x, train_y)
    pass

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

RF average r2: 0.5298487250690131, score_list r2: [0.12211244 0.47979459 0.4747057  0.83443889 0.57864971 0.6264231
 0.59281665]


 33%|████████████████████████████                                                        | 1/3 [00:38<01:16, 38.17s/it]

RF average neg_mean_squared_error: 846.035591367235, score_list neg_mean_squared_error: [ 489.75978695 1084.3894782   957.10604299  969.15394293  535.46026365
  633.70294246 1252.6766824 ]
model_xgb average r2: 0.5638275837382697, score_list r2: [-0.12210711  0.46468874  0.57392184  0.79115288  0.66787072  0.75801346
  0.81325256]


 67%|████████████████████████████████████████████████████████                            | 2/3 [02:09<01:09, 69.51s/it]

model_xgb average neg_mean_squared_error: 780.4033199962438, score_list neg_mean_squared_error: [ 580.52271664 1076.34280391  856.10715144 1095.05554964  486.67142537
  533.54277843  834.58081455]
model_lgb average r2: 0.6043437890622627, score_list r2: [0.11092336 0.54748228 0.61985654 0.81565424 0.61518892 0.7128994
 0.80840178]


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:15<00:00, 45.29s/it]

model_lgb average neg_mean_squared_error: 756.3094815575902, score_list neg_mean_squared_error: [ 516.73950293  989.61259384  808.64378353 1028.81782958  523.84891197
  581.15327491  845.35047415]


## 변수 중요도 확인

In [51]:
model_list2 = [
                ('RF', RandomForestRegressor(n_estimators=5)),
                ('model_xgb', xgb.XGBRegressor()),
                ('model_lgb', lgb.LGBMRegressor())
            ]

In [52]:
# 피팅된 모델에 따른 변수 중요도

def plot_feature_importance(model):
    name = model_tuple[0]
    model = model_tuple[1]
    n_features = preprocessed_train_df.shape[1]
    plt.barh(np.arange(n_features), sorted(model.feature_importances_), align="center")
    plt.yticks(np.arange(n_features), preprocessed_train_df.columns)
    plt.xlabel("Feature Importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)
    

In [57]:
plot_feature_importance(RandomForestRegressor(n_estimators=5).fit(train_X,train_y))

AttributeError: 'LinearRegression' object has no attribute 'feature_importances_'

### 3. 파라미터 튜닝

In [7]:
# 3. test get_best_param function

# linear regression model object
def linear_object(trial:Trial, data, target):
    params = {
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False])
    }
    
    test_model = LinearRegression(**params)
    test_model_score = execute_modeling(('LR', test_model), data, target)

    return test_model_score

# xgbr regression model object
def xgbr_object(trial:Trial, data, target):
    params = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 1000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 100),
        'gamma':trial.suggest_int('gamma', 1, 3),
        'learning_rate': 0.01,
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 1.0),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    
    test_model = xgb.XGBRegressor(**params)
    test_model_score = execute_modeling(('XGBR', test_model), data, target)

    return test_model_score

# light gbm regression model object
def lgb_object(trial:Trial, data, target):
    params = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 1000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 100)
    }
    
    test_model = lgb.LGBMRegressor(**params)
    test_model_score = execute_modeling(('LGBM', test_model), data, target)

    return test_model_score

# 하이퍼 파라미터 결과 도출
# 위에서 분할한  X_train, y_train, X_test, y_test 파라미터 삽입
study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(lambda trial: lgb_object(trial, train_x, train_y), n_trials=3)

best_score = study.best_value
best_param_dict = study.best_trial.params

print(best_score, best_param_dict)

[I 2021-12-28 16:21:42,984] A new study created in memory with name: no-name-aa9728d7-5b52-4b7f-a314-9ab0fc9daa4f
[I 2021-12-28 16:22:21,732] Trial 0 finished with value: 25895.329099756465 and parameters: {'n_estimators': 790, 'max_depth': 14, 'min_child_weight': 43}. Best is trial 0 with value: 25895.329099756465.


LGBM average rmse_scores: 25895.329099756465, rmse_scores: [19576.06321571 19376.58158036 55598.44322939 56276.78036999
 29394.86794348 12642.15597452 12019.52356509 25045.80398683
 21638.95337171 34075.15963913 14420.76480632 13809.82873753
 21766.47121174 26651.99570745 26136.5431571 ]


[I 2021-12-28 16:23:02,077] Trial 1 finished with value: 26214.330928425254 and parameters: {'n_estimators': 750, 'max_depth': 9, 'min_child_weight': 64}. Best is trial 0 with value: 25895.329099756465.


LGBM average rmse_scores: 26214.330928425254, rmse_scores: [19831.40428064 20039.21382514 54863.88110247 56009.72590423
 29900.24350841 13507.73394071 12359.79148079 25557.985213
 22598.22387112 34117.80646005 14865.52649845 13909.8459276
 21980.87217221 27063.63479704 26609.07494452]


[I 2021-12-28 16:23:40,147] Trial 2 finished with value: 25739.27317819786 and parameters: {'n_estimators': 727, 'max_depth': 14, 'min_child_weight': 21}. Best is trial 2 with value: 25739.27317819786.


LGBM average rmse_scores: 25739.27317819786, rmse_scores: [19982.4711522  19401.4453833  55313.02189458 56188.8881972
 29589.53745901 12440.62745936 11658.56881514 24586.18736491
 21635.68308552 33538.90012906 14624.16575483 13800.14089515
 21483.4692944  26478.39201484 25367.59877348]
25739.27317819786 {'n_estimators': 727, 'max_depth': 14, 'min_child_weight': 21}


In [8]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [9]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

### 최종 모형 검증

In [19]:
# 각 모델에 대하여 실질적 학습 수행
# execute_modeling 함수에서 실행한 결과에서 성능 가장 좋은 모델 선정하여 검증 ㄱㄱ
model = xgb.XGBRegressor(random_state=1)

clf = model.fit(train_x, train_y)
pred = clf.predict(test_x)
# print(pred)

execute_modeling(('model_xgb', clf), test_x, test_y)


# 희섭님 
# model = xgb.XGBRegressor()

# clf = model.fit(train_x, train_y)
# pred = clf.predict(test_x)
# # print(pred)

# from sklearn.metrics import mean_squared_error, r2_score
# mse = mean_squared_error(test_y, pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(test_y, pred)

# print('RMSE: ',rmse)
# print('R^2: ', r2 )

[1413.8854 1551.0321 1423.9518 ... 1473.5942 1499.4089 1499.1306]
model_xgb average r2: 0.0342141368984501, score_list r2: [-0.31593511 -0.26629261  0.06465993  0.42507882 -0.57291284  0.17314199
  0.73175878]
model_xgb average neg_mean_squared_error: 1527.3000992759873, score_list neg_mean_squared_error: [2734.9881029  1170.07021469  794.16494052  942.35014868 2630.98610101
 1698.27286712  720.26832001]


1527.3000992759873

In [ ]:
'''
# 기본데이터 + 8학군 포함
   
    *모델 score*
    *검증 score*
    
    
    *모델 score* -> 8학군
LR average r2: -2.0420180842797835e+25, score_list r2: [-3.90509049e+25 -3.38168182e+24 -8.29300556e+25 -1.75786236e+25
  2.52087524e-01  4.06268747e-01  3.43976035e-01]
 25%|█████████████████████                                                               | 1/4 [00:01<00:05,  1.95s/it]
LR average neg_mean_squared_error: 4017165228386499.0, score_list neg_mean_squared_error: [3.42466120e+15 2.70528978e+15 1.19437160e+16 1.00464896e+16
 7.30310680e+02 8.35735010e+02 1.56423111e+03]
RF average r2: 0.5615398859552887, score_list r2: [0.15291021 0.46948403 0.49229745 0.82369328 0.55673155 0.65698717
 0.7786755 ]
 50%|██████████████████████████████████████████                                          | 2/4 [00:21<00:25, 12.59s/it]
RF average neg_mean_squared_error: 810.0775830404102, score_list neg_mean_squared_error: [ 511.07339987 1139.47022893  939.27327296  982.2944832   552.65982839
  633.04444021  912.72742771]
model_xgb average r2: 0.5705402557855545, score_list r2: [0.25699085 0.47258897 0.4825857  0.82797677 0.55035536 0.63427545
 0.7690087 ]
 75%|███████████████████████████████████████████████████████████████                     | 3/4 [01:29<00:37, 37.53s/it]
model_xgb average neg_mean_squared_error: 804.0552988402005, score_list neg_mean_squared_error: [ 472.38831129 1068.37083928  943.41483927  993.83768738  566.26142199
  655.91973462  928.19425806]
model_lgb average r2: 0.5713493564077509, score_list r2: [0.25051025 0.55240923 0.57711379 0.81984651 0.47244313 0.59279706
 0.73432552]
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:34<00:00, 23.75s/it]
model_lgb average neg_mean_squared_error: 804.2173492013026, score_list neg_mean_squared_error: [ 474.44394744  984.21047015  852.89437663 1017.05221558  613.36205134
  692.11632538  995.44205789]
  
   *검증 score*
LGBM average rmse_scores: 25895.329099756465, rmse_scores: [19576.06321571 19376.58158036 55598.44322939 56276.78036999
 29394.86794348 12642.15597452 12019.52356509 25045.80398683
 21638.95337171 34075.15963913 14420.76480632 13809.82873753
 21766.47121174 26651.99570745 26136.5431571 ]
[I 2021-12-28 16:23:02,077] Trial 1 finished with value: 26214.330928425254 and parameters: {'n_estimators': 750, 'max_depth': 9, 'min_child_weight': 64}. Best is trial 0 with value: 25895.329099756465.
LGBM average rmse_scores: 26214.330928425254, rmse_scores: [19831.40428064 20039.21382514 54863.88110247 56009.72590423
 29900.24350841 13507.73394071 12359.79148079 25557.985213
 22598.22387112 34117.80646005 14865.52649845 13909.8459276
 21980.87217221 27063.63479704 26609.07494452]
[I 2021-12-28 16:23:40,147] Trial 2 finished with value: 25739.27317819786 and parameters: {'n_estimators': 727, 'max_depth': 14, 'min_child_weight': 21}. Best is trial 2 with value: 25739.27317819786.
LGBM average rmse_scores: 25739.27317819786, rmse_scores: [19982.4711522  19401.4453833  55313.02189458 56188.8881972
 29589.53745901 12440.62745936 11658.56881514 24586.18736491
 21635.68308552 33538.90012906 14624.16575483 13800.14089515
 21483.4692944  26478.39201484 25367.59877348]
25739.27317819786 {'n_estimators': 727, 'max_depth': 14, 'min_child_weight': 21}

'''